In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import numpy as np 
import pandas as pd 
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
%matplotlib inline

In [2]:
print(os.getcwd())
os.chdir('/Users/liuyang/Desktop/Springboard_Capstone3') 
print(os.getcwd())

/Users/liuyang/Desktop/Springboard_Capstone3/Notebook
/Users/liuyang/Desktop/Springboard_Capstone3


In [3]:
df = pd.read_csv('data/train.csv')

In [4]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [5]:
df.shape

(1017209, 9)

In [6]:
df = df.sort_values(['Date', 'Store'], ascending=True)

In [7]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
1016095,1,2,2013-01-01,0,0,0,0,a,1
1016096,2,2,2013-01-01,0,0,0,0,a,1
1016097,3,2,2013-01-01,0,0,0,0,a,1
1016098,4,2,2013-01-01,0,0,0,0,a,1
1016099,5,2,2013-01-01,0,0,0,0,a,1


In [8]:
# class data_process:
    
#     def __init__(self, data):
#         self.data = data
#         self.count = data.nunique()
#         self.va_pct = round(self.count/(data.shape[0])*100,4)
#         self.nan_p = nan_p = round(data.isnull().sum()/data.shape[0]*100,4)
        
    
#     def data_info(self):
#         frame = pd.DataFrame(zip(self.count,self.va_pct,self.nan_p), 
#                              index=self.count.index, 
#         columns=['counts','unique_value_pct', 'nan_pct']).reset_index().rename(columns={'index': 'column'})

#         frame['data_type']= self.data.dtypes.tolist()
#         return frame.sort_values('counts', ascending=False)
   

#     def convert_datetime(self, datecol):
#         self.data[datecol] = pd.to_datetime(self.data[datecol], format='%Y-%m-%d')
    
# #     def int_to_str(self, col):
# #         self.data[col] = self.data[col].astype('str')
    
#     def convert_dtype(self):
#         catcolumns = self.data.select_dtypes(include=['object']).columns.tolist()
#         return pd.get_dummies(self.data, columns=catcolumns)
    
    
#     def pivot_table(self, index, aggfunc):
#         return pd.pivot_table(self.data, index=index, aggfunc=aggfunc)

In [9]:
# class Sales_EDA:
#     def __init__(self, data):
#         self.data=data
    
        
#     def nlargest_ID(self, N):
#         topN_storeID = self.data.groupby('Store')['Sales'].sum().nlargest(N).index.tolist()
#         return topN_storeID
    
#     def nsmallest_ID(self, N):
#         bottomN_storeID = self.data.groupby('Store')['Sales'].sum().nsmallest(N).index.tolist()
#         return bottomN_storeID
    
#     def topN_weekly_salesplot(self, N):
#         plt.figure(figsize=(12, 7))
#         for storeid in self.nlargest_ID(N):
#             self.data.loc[self.data.Store==storeid].set_index('Date').Sales.resample('w').sum().plot(label= f'Store {storeid}')
#         plt.legend()
#         plt.title(f'Weekly Sales of Top {N} Store', fontsize=15)
#         plt.xlabel('Date', fontsize=15)
#         plt.ylabel('Weekly Sales', fontsize=15)
#         plt.show()
        
#     def bottomN_weekly_salesplot(self, N):
#         plt.figure(figsize=(12, 7))
#         for storeid in self.nsmallest_ID(N):
#             self.data.loc[self.data.Store==storeid].set_index('Date').Sales.resample('w').sum().plot(label= f'Store {storeid}')
#         plt.legend()
#         plt.title(f'Weekly Sales of Bottom {N} Store', fontsize=15)
#         plt.xlabel('Date', fontsize=15)
#         plt.ylabel('Weekly Sales', fontsize=15)
#         plt.show()
    
    
#     def topN_daily_violinplot(self, N):
#         plt.figure(figsize=(12, 7))
#         N_data = self.data.loc[self.data.Store.isin(self.nlargest_ID(N))]
#         sns.violinplot(x='Sales', y='Store', 
#                        data=N_data, order=self.nlargest_ID(N),
#                       orient='h')
#         plt.title('Daily Sales of Top10 Stores', fontsize=15)
#         plt.xlabel('Sales', fontsize=15)
#         plt.ylabel('Store', fontsize=15)
#         plt.show()

#     def bottomN_daily_violinplot(self, N):
#         plt.figure(figsize=(12, 7))
#         b_N_data = self.data.loc[self.data.Store.isin(self.nsmallest_ID(N))]
#         sns.violinplot(x='Sales', y='Store', 
#                        data=b_N_data, order=self.nsmallest_ID(N),
#                       orient='h')
        
#         #sns.violinplot(x='Store', y='Sales', data=df.loc[df.Store.isin(self.nsmallest_ID(N))])
#         plt.title('Daily Sales of Bottom Stores', fontsize=15)
#         plt.xlabel('Daily Sales', fontsize=15)
#         plt.ylabel('Store ID', fontsize=15)
#         plt.show()

In [10]:
# class sumby:
#     def __init__(self, data, groupbycol, aggcol):
#         self.data = data
#         self.groupbycol = groupbycol
#         self.aggcol = aggcol
       
        
#     def get_sum_pct(self):
#         total = self.data.groupby(self.groupbycol)[self.aggcol].sum()
#         total = pd.DataFrame(total)
#         total['pct'] = (total[self.aggcol]/total[self.aggcol].sum())*100
#         total = total.sort_values(self.aggcol, ascending=False)
#         total = total.reset_index()
#         return total
    
#     def histgram(self,col):
#         self.get_sum_pct()[col].hist()
#         per50 = self.get_sum_pct()[col].describe()['50%']
#         mean = self.get_sum_pct()[col].describe()['mean']
#         plt.axvline(per50,color='r')
#         plt.axvline(mean, color='r', linestyle='-.')
#         plt.legend(['median', 'mean'])
#         plt.title(f'Total {self.aggcol} by {self.groupbycol}', fontsize=15)
#         plt.show()
    
#     def topN_barplot(self, N):
#         bardata = self.get_sum_pct().set_index(self.groupbycol)
#         plt.figure(figsize=(12,7))
#         sns.barplot(y = bardata.Sales.nlargest(N).index, 
#                     x = bardata.Sales.nlargest(N).values, 
#                     order = bardata.Sales.nlargest(N).index,
#                    orient='h')            
#         plt.title(f'Total {self.aggcol} of Top {N} Stores', fontsize=15)
#         plt.xlabel(self.aggcol, fontsize=15)
#         plt.ylabel(self.groupbycol, fontsize=15)
    
#     def topN_sum_share(self, N):
#         top10sum = round(self.get_sum_pct().iloc[:N].sum()[1],0)
#         top10pct = round(self.get_sum_pct().iloc[:N].sum()[2],3)
#         if len(self.data) > 1115:
#             print(f'Top {N} {self.groupbycol} Total {self.aggcol}: {top10sum}')
#             print(f'Top {N} {self.groupbycol} Total {self.aggcol} Share in All Stores: {top10pct}%')
#         else:
#             storetype = list(set(self.data.StoreType))[0].upper()
#             print(f'{storetype} Type Store Top {N} {self.groupbycol} Total {self.aggcol}: {top10sum}')
#             print(f'{storetype} Type Store Top {N} {self.groupbycol} Total {self.aggcol} Share: {top10pct}%')

In [11]:
# class individual_store:
#     def __init__(self, data, indexcol, storeid):
#         self.data = data
#         self.indexcol = indexcol
#         self.storeid=storeid
#         self.total_sales = sumby(self.data,'Store', 'Sales').get_sum_pct()
#         self.id_total = self.total_sales.loc[self.total_sales.Store==self.storeid]
#         self.storesales = self.id_total.iloc[0][1]
#         self.ranking = self.id_total.index[0]+1
#         self.share = round(self.id_total.iloc[0][2],3)
#         self.opendays = self.data.loc[self.data.Store==self.storeid].set_index(self.indexcol).Open.value_counts()[1]
#         if len(self.data.loc[self.data.Store==self.storeid].set_index(self.indexcol).Open.value_counts().index)==2:
#             self.closeddays = self.data.loc[self.data.Store==self.storeid].set_index(self.indexcol).Open.value_counts()[0]
#         else:
#             self.closeddays=0
 
#     def store_opendays(self):
#         print(f'Store {self.storeid} open days: {self.opendays}')
#         print(f'Store {self.storeid} closed days: {self.closeddays}')

            
#     def get_sales_share_ranking(self):
#         storenumber = len(set(self.data.Store))
#         print(f'Total Sales of Store {self.storeid} is {self.storesales}')
#         print(f'Sales share among {storenumber} stores : {self.share}%')
#         print(f'Ranking among {storenumber} stores : {self.ranking}')
     
#     def lineplot(self,col):
#         plt.figure(figsize=(12, 7))
#         dailydata = self.data.loc[self.data.Store==self.storeid].set_index('Date')[col]
#         plt.plot(dailydata)
#         plt.plot(dailydata.index,dailydata.rolling(14).mean(), 'red')
#         plt.plot(dailydata.resample('w').sum(), 'green')
#         plt.legend([f'Daily {col}', 'Rolling mean (14D)', f'Weekly {col}'])
#         plt.title(f'{col} of store {self.storeid}', fontsize=15)
#         plt.show()
        
#     def store_report(self):
#         storetype = newstoredata.loc[newstoredata.Store==self.storeid].StoreType.iloc[0]
#         assortmenttype = newstoredata.loc[newstoredata.Store==self.storeid].Assortment.iloc[0]
#         storetypedata = sumby(newstoredata.loc[newstoredata.StoreType==storetype],'Store', 'Sales').get_sum_pct()
#         assortttypedata = sumby(newstoredata.loc[newstoredata.Assortment==assortmenttype],'Store', 'Sales').get_sum_pct()
#         ranking2 = storetypedata.loc[storetypedata.Store==self.storeid].index[0]+1
#         ranking3 = assortttypedata.loc[assortttypedata.Store==self.storeid].index[0]+1
#         storenumber2 = len(storetypedata)
#         storenumber3 = len(assortttypedata)
#         share2 = round(storetypedata.iloc[0][2],3)
#         share3 = round(assortttypedata.iloc[0][2],3)
                                        
        
#         return pd.DataFrame([self.storeid,storetype.upper(),assortmenttype.upper(), self.opendays, 
#                              self.closeddays, self.storesales,self.share, share2, share3, 
#                              self.ranking, ranking2, ranking3],
#                            index=['Storeid', 'Store Type', 'Assortment', 'Open Days',
#                                  'Closed Days', 'Total Sales', 'Share in All Store (%)', 
#                                  f'Share in {storetype.upper()} Type (%)', 
#                                  f'Share in {assortmenttype.upper()} Assortment (%)',
#                                  'Ranking in All Store',
#                                  f'Ranking in {storetype.upper()} Type', 
#                                  f'Ranking in {assortmenttype.upper()} Assortment'],
#                            columns=['Detail'])
    
        

In [15]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/Users/liuyang/Desktop/Springboard_Capstone3/notebook')

import eda

In [ ]:
from eda import data_process, 
Sales_EDA, 
sumby, 
individual_store, 
combined_storedata, 
combined_store_EDA

In [ ]:
df_process = data_process(df)

In [ ]:
df_process.convert_datetime('Date')

In [ ]:
df_process.data_info()

In [ ]:
df.describe().T

In [ ]:
period_days = len(set(df.Date))
period_weeks = len(df.set_index('Date').groupby('Date').resample('w'))
numberOfstore = len(set(df.Store))

print(f'Data contains data of {numberOfstore} stores')
print(f'Data contains data of {period_days} days (2013-01-01 ~ 2015-07-31)')
print(f'Data contains data of {period_weeks} weeks')

## Daily Sales

In [ ]:
def histplot_mean_median(data, col):
    data[col].hist()
    per50 = data[col].describe()['50%']
    mean = data[col].describe()['mean']
    plt.axvline(per50,color='r')
    plt.axvline(mean, color='r', linestyle='-.')
    plt.legend(['median', 'mean'])

    plt.title(f'Daily {col} Distribution', fontsize=15)
    plt.show()

In [ ]:
# check daily sales of 1115 stores over 942 days
plt.rcParams['figure.figsize'] = (12, 7)
histplot_mean_median(df, 'Sales')

sns.boxplot(x=df.Sales)
plt.title('Daily Sales', fontsize=15)
plt.show()

In the histogram, we can see mean and median are very close to each other

In [ ]:
# check the details of daily sales outlier
df.loc[df.Sales.idxmax()]

In [ ]:
# check store open days and sales plot for store 909
storeid=909
store909 = individual_store(df, 'Date', storeid)

store909.store_opendays()
store909.lineplot('Sales')

We can see the sales peak happens at 2015-06-22.
By analyzing the sales plot of store 909 we can see the peak was just an one time exception.

## Daily Customer

In [ ]:
# check daily customers of 1115 stores over 942 days
histplot_mean_median(df, 'Customers')

sns.boxplot(x=df.Customers)
plt.title('Daily Customers', fontsize=15)
plt.show()

In [ ]:
# check the details of daily customers outlier
df.loc[df.Customers.idxmax()]

In [ ]:
# check the customer line plot for store 817
storeid=817
store817 = individual_store(df, 'Date',storeid)
store817.store_opendays()
store817.lineplot('Customers')

We can see the peak of customer happens at 2013-01-22. By analyzing the customer plot of store 817 we can see the peak was just an one time exception.

## 942 Days Sum up by Store

In [ ]:
sumsales_by_store = sumby(df, 'Store', 'Sales')

In [ ]:
#alldata = totalby(df, 'Date', 'Customers')

In [ ]:
#alldata.total_by()

In [ ]:
# make a dataframe contains sum up sales and total sales shares for each store over 942 days
# sorted in descending order
sumsales_by_store.get_sum_pct()

In [ ]:
# statistic summary of total sales and shares of each store over 942 days
sumsales_by_store.get_sum_pct().describe()

In [ ]:
# check distribution of sum up sales over 942 days of 1115 stores
sumsales_by_store.histgram('Sales')

sns.boxplot(x=sumsales_by_store.get_sum_pct().Sales)
plt.show()

In [ ]:
# get the sum of sales and share of top 10 stores over 942 day
sumsales_by_store.topN_sum_share(10)

## Sum up Sales Barplot of Top 10 Stores over 942 Days

In [ ]:
sumsales_by_store.topN_barplot(10)

## Daily Sales Violinplot of Top 10 Stores over 942 Days

In [ ]:
# check daily sales of Top 10 stores 
sales_by_store= Sales_EDA(df)
sales_by_store.topN_daily_violinplot(10)

## Weekly Total Sales

In [ ]:
markers= df_process.pivot_table('Date', 'sum').resample('w').Sales.sum().nlargest(3)
weeklytotalsales = df_process.pivot_table('Date', 'sum').resample('w').Sales.sum()

plt.figure(figsize=(12, 7))
plt.plot(weeklytotalsales.index, weeklytotalsales.values)
plt.plot(markers.index, markers.values, "D", markersize=8)
plt.title('Weekly Sales of All Stores', fontsize=15)
plt.xlabel('Date', fontsize=15)
plt.ylabel('Weekly Sales', fontsize=15)
plt.show()

print(f'Three peaks happens at: \n {markers}')

#### From the weekly sales sum up by all stores, We can see the 3 peaks happens at 12-08 ~ 12-22 week in both 2013 and 2014, which is around christmas season. Therefore we assume this trend also exist in individual store. 

## Plot the number of daily open stores over 942 days

In [ ]:
pd.pivot_table(df[['Date', 'Store']], index='Date', aggfunc='count').plot()
plt.title('Number of Open Stores (Daily)')
plt.show()

In [ ]:
closedstore = (pd.pivot_table(df[['Date', 'Store']], index='Date', aggfunc='count').max() - pd.pivot_table(df[['Date', 'Store']], index='Date', aggfunc='count').min())[0]
print(f'number of closed store:{closedstore}')

#### We can see there's a big decline in sales during 2014-07-01 ~ 2014-12-31 as 180 stores were closed due to refurbishment

## Check Weekly Sales Plot for Top 3 and Bottom 3 Stores

In [ ]:
sales_by_store.topN_weekly_salesplot(3)
sales_by_store.bottomN_weekly_salesplot(3)

## Check Weekly Sales Plot for Top 10 and Bottom 10 Stores

In [ ]:
sales_by_store.topN_weekly_salesplot(10)
sales_by_store.bottomN_weekly_salesplot(10)

## Conclusion 1: we can see peak during christmas season is common among most of stores 

### Add 3 new metrics: AverageDailySales, AverageDailyCustomer, SalesPerCustomer 

In [ ]:
store_sum = df_process.pivot_table('Store', 'sum')
store_sum['AverageDailySales'] = store_sum['Sales']/store_sum['Open']
store_sum['AverageDailyCustomer'] = store_sum['Customers']/store_sum['Open']
store_sum['SalesPerCustomer'] = store_sum['Sales']/store_sum['Customers']

In [ ]:
store_sum.sort_values('Sales', ascending=False)

## Average Daily Sales

In [ ]:
# plot 10 stores with the highest avarege daily sales

sns.barplot(y=store_sum.AverageDailySales.nlargest(10).index,
            x=store_sum.AverageDailySales.nlargest(10).values,
            order = store_sum.AverageDailySales.nlargest(10).index,
           orient='h')

plt.title('Average Daily Sales Top 10 Store', fontsize=15)
plt.xlabel('AverageDailySales', fontsize=15)
plt.ylabel('Store', fontsize=15)
plt.show()

In [ ]:
# # check the ranking of sales per customer of store 262, store 817
# newstoredata.SalesPerCustomer[261]
# print(sorted(newstoredata.SalesPerCustomer).index(newstoredata.SalesPerCustomer[261]))

# newstoredata.SalesPerCustomer[816]
# print(sorted(newstoredata.SalesPerCustomer).index(newstoredata.SalesPerCustomer[816]))

# newstoredata.SalesPerCustomer[1113]
# print(sorted(newstoredata.SalesPerCustomer).index(newstoredata.SalesPerCustomer[1113]))

# Note: If we can increase the sales per customer for store 262, 817, 34, the impact on increasing total sales will be big.


In [ ]:
# compare it with plot of 10 highest total sales (sum of 942 days)

sumsales_by_store.topN_barplot(10)

In [ ]:
set(sales_by_store.nlargest_ID(10)).difference(set(store_sum.AverageDailySales.nlargest(10).index))

In [ ]:
set(store_sum.AverageDailySales.nlargest(10).index).difference(set(sales_by_store.nlargest_ID(10)))

We can see the top 10 stores in average daily sales ranking plot are very similar to top 10 stores in total sales ranking, except store 842 replaced store 733 in average daily sales ranking. 

## Average Daily Customers 

In [ ]:
sns.barplot(y=store_sum.AverageDailyCustomer.nlargest(10).index,
            x=store_sum.AverageDailyCustomer.nlargest(10).values,
            order = store_sum.AverageDailyCustomer.nlargest(10).index,
           orient='h')

plt.title('Top 10 Ranking of Average Daily Customers', fontsize=15)
plt.xlabel('AverageDailyCustomer', fontsize=15)
plt.ylabel('Store', fontsize=15)
plt.show()

## Sales Per Customer 

In [ ]:
sns.barplot(y=store_sum.SalesPerCustomer.nlargest(10).index,
            x=store_sum.SalesPerCustomer.nlargest(10).values,
            order = store_sum.SalesPerCustomer.nlargest(10).index,
           orient='h')

plt.title('Top 10 Ranking of Sales Per Customer', fontsize=15)
plt.xlabel('SalesPerCustomer', fontsize=15)
plt.ylabel('Store', fontsize=15)
plt.show()